In [ ]:
# Necessary Libraries
import pandas as pd
import numpy as np
import networkx as nx
import json
from pprint import pprint

In [ ]:
# Helper maps/dictionaries

# Map from string to its numerical weight
edge_weight_map = { '++': 2, '+': 1, '0': 0, '-': -1, '--': -2 }

# Data type of edge in graph
dt_edge = [('weight', int), ('weight_absolute', int), ('sign', int)]

# Maps string to dt_edge tripple 
def edge_map(w):
    w = edge_weight_map[w]
    return (w, abs(w), np.sign(w))

In [ ]:
# Extracts action systems from loaded data
def get_action_systems(data):
    entries = {}
    
    for i, entry in enumerate(data):
        entries[entry['id']] = entry['name']
    
    return entries

def get_life_entries(data, action_systems):
    names = {}
    node_action_systems = {}
    influences = {}
    
    for i, entry in enumerate(data):
        names[i] = entry['title']
        node_action_systems[i] = action_systems[entry['actionSystemId']]
        influences[i] = entry['influence']
    
    return names, node_action_systems, influences

# Loads the json and creates a networkx multidi graph
def json_to_graph(path, encoding='utf-8', verbose=False):
    json_data = None
    action_systems = {}
    names = {}
    node_action_systems = {}
    influences = {}
    
    with open(path, encoding=encoding) as data_file:
        json_data = json.load(data_file)
    
    # Print the loaded json data
    if verbose == True:
        ppriint(json_data)
    
    # Gets adjacency matrix and maps it to the dt_edge data type
    df = pd.DataFrame(json_data['connections'])
    df = df.applymap(edge_map)
    am = np.array(df, dtype=dt_edge)
    
    if verbose == True:
        # Print mapped adjacency matrix
        print(am)
    
    # Get action systems
    action_systems = get_action_systems(json_data['actionSystems'])
    
    # Get life entries
    names, node_action_systems, influences = get_life_entries(json_data['lifeEntries'], action_systems)
    
    # Create graph by loading the adjacency matrix
    G = nx.from_numpy_matrix(am, parallel_edges=False, create_using=nx.MultiDiGraph())
    
    # Set attributes on nodes
    nx.set_node_attributes(G, names, name='name')
    nx.set_node_attributes(G, node_action_systems, name='actionSystem')
    nx.set_node_attributes(G, influences, name='influence')
    
    return G